In [13]:
from arcgis.gis import GIS
import pandas as pd

gis = GIS("https://www.jdart.uscourts.gov/arcgis/", "ao_admin", "4D~?Lv2qrsa5=#dc")

In [14]:
find_id = input('ItemID: ')

find_url = gis.content.get(find_id).url if find_id else input('Service URL: ')
print(find_url)

https://www.jdart-hst.uscourts.gov/arcgis/rest/services/Hosted/PowerOutageUS_City/FeatureServer


In [15]:
webmaps = gis.content.search('', item_type='Web Map', max_items=-1, outside_org=False)

map_list = [m for m in webmaps if str(m.get_data()).find(find_url) > -1]

In [16]:
webmaps

[<Item title:"JDART - Middle District of Georgia (GAM) Alert Status Web Map" type:Web Map owner:f47c0274_123c_4320_8742_fb117e700e11>,
 <Item title:"Indoors DC.TMFJB.3" type:Web Map owner:f47c0274_123c_4320_8742_fb117e700e11>,
 <Item title:"MSS - Public View - Survey Reporting - Web Map" type:Web Map owner:f47c0274_123c_4320_8742_fb117e700e11>,
 <Item title:"GUX - Survey Review App - Web Map" type:Web Map owner:f47c0274_123c_4320_8742_fb117e700e11>,
 <Item title:"LAE - Survey Review App - Web Map" type:Web Map owner:f47c0274_123c_4320_8742_fb117e700e11>,
 <Item title:"Indoors DC.TMFJB.6" type:Web Map owner:f47c0274_123c_4320_8742_fb117e700e11>,
 <Item title:"HIX - Public View - Survey Reporting - Web Map" type:Web Map owner:f47c0274_123c_4320_8742_fb117e700e11>,
 <Item title:"VIX - Public View - Survey Reporting - Web Map" type:Web Map owner:f47c0274_123c_4320_8742_fb117e700e11>,
 <Item title:"Indoors DC.TMFJB.G" type:Web Map owner:f47c0274_123c_4320_8742_fb117e700e11>,
 <Item title:"A

In [17]:
apptypes = ['Application', 'Dashboard', 'Story Map', 'Web Experience']

webapps = [item for sublist in [gis.content.search('', item_type=t, max_items=-1, outside_org=False) for t in apptypes] for item in sublist]

In [18]:
app_list = []

for w in webapps:
    
    try:
        wdata = str(w.get_data())

        criteria = [
            wdata.find(find_url) > -1,
            wdata.find(find_id) > -1,
            any([wdata.find(m.id) > -1 for m in map_list])
        ]
        
        if any(criteria):
            app_list.append(w)
    
    # Some apps don't have data, so we'll just skip them if they throw a TypeError
    except:
        continue

In [19]:
dependencies = pd.concat(
    [
        pd.DataFrame([{'title':a.title, 'id':a.id, 'type':a.type, 'url':f'{gis.url}/home/item.html?id={a.id}', 'data':a.get_data()} for a in app_list]),
        pd.DataFrame([{'title':m.title, 'id':m.id, 'type':m.type, 'url':f'{gis.url}/home/item.html?id={m.id}', 'data':m.get_data()} for m in map_list])
    ]
)

dependencies.reset_index(inplace=True, drop=True)

dependencies.head()

,title,id,type,url,data
0,Severe Weather Map Viewer,d83a5471af99487f89696645300b226a,Web Map,https://www.jdart.uscourts.gov/arcgis//home/it...,{'operationalLayers': [{'id': '18a701356a6-lay...


In [20]:
dependencies

,title,id,type,url,data
0,Severe Weather Map Viewer,d83a5471af99487f89696645300b226a,Web Map,https://www.jdart.uscourts.gov/arcgis//home/it...,{'operationalLayers': [{'id': '18a701356a6-lay...


In [21]:
import csv
from datetime import datetime
csv_file = r"C:\Users\bob10704\OneDrive - Esri\Desktop\Projects\1.Active Projects\US Courts\POUS_Items_Prod.csv"

In [22]:
now = datetime.now()
dt_string = now.strftime("%Y%m%d_%H%M%S")

In [23]:
with open(csv_file.format(dt_string), 'w') as csvfile:
    dataWriter = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL, lineterminator='\n')
    dataWriter.writerow(['Title',
                         'Item ID',
                         'Item Type'])
    for item in webapps:
        dataWriter.writerow([
            item.title,
            item.id,
            item.type
        ])
    for item in webmaps:
        dataWriter.writerow([
            item.title,
            item.id,
            item.type
        ])